<a href="https://colab.research.google.com/github/jam2466/JulianMcWilliams-github.io/blob/main/stock%20analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Only run this code if you are getting issues with the chart below
!pip install --upgrade plotly

In [3]:
# Import required libraries
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd

# Define the candlestick plotting function
def candlestick(ticker, start_date, end_date, sma1, sma2):
    df = yf.download(ticker, start=start_date, end=end_date)
    # Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    # Calculate SMAs
    df['SMA1'] = df['Close'].rolling(window=sma1).mean()
    df['SMA2'] = df['Close'].rolling(window=sma2).mean()
    return df

def plot_candlestick(df):
    # Plot
    fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick'
    )])
    fig.add_trace(go.Scatter(
        x=df.index, y=df['SMA1'],
        mode='lines', name=f'SMA {sma1}', line=dict(color='magenta')
    ))
    fig.add_trace(go.Scatter(
        x=df.index, y=df['SMA2'],
        mode='lines', name=f'SMA {sma2}', line=dict(color='green')
    ))
    fig.update_layout(
        title=f'{ticker} Candlestick Chart with SMA {sma1} and SMA {sma2}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False
    )
    fig.show()

In [ ]:
#These SMs yield the highest profit if you buy/sell on the intersection days, calculated below
#for date range: start_date = '2022-05-20', end_date = '2025-05-20'

#LLY: sm1=7, sma2=14
#MSFT: sm1=16, sm2=27
#SPY: sm1=24, sm2=59
#CMG: sm1=7, sm2=89
#MOD: sm1=29, sm2=96
#TSLA: sm1=9, sm2=15

In [5]:
ticker = 'TSLA'
start_date = '2022-05-20'
end_date = '2025-05-20'
sma1 = 9
sma2 = 15
df = candlestick(ticker, start_date, end_date, sma1, sma2)
plot_candlestick(df)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
def profit(df):
  #buy/sell on dates where SMA1 and SMA2 intersect
  buy = df[(df["SMA1"] > df["SMA2"]) & (df["SMA1"].shift(1) < df["SMA2"].shift(1))]
  sell = df[(df["SMA1"] < df["SMA2"]) & (df["SMA1"].shift(1) > df["SMA2"].shift(1))]
  #assume worst case for trading in a day
  profit = sell.High.sum() - buy.Low.sum()
  return profit

In [6]:
#now cycle through multiple sma pairs to find highest profit
def optimal_sma(ticker, start_date, end_date):
  current_max_profit = 0
  max_sma1 = 0
  max_sma2 = 0
  for sma1 in range(7, 30):
      for sma2 in range(15, 100):
          if sma2/sma1 < 1.25:
              continue
          try:
            df = candlestick(ticker, start_date, end_date, sma1, sma2)
            current_profit = profit(df)
          except:
            continue
          if current_profit > current_max_profit and current_profit is not None and not pd.isna(current_profit):
              current_max_profit = current_profit
              current_max_sma1 = sma1
              current_max_sma2 = sma2
  max_profit = current_max_profit
  max_sma1 = current_max_sma1
  max_sma2 = current_max_sma2
  return max_profit, max_sma1, max_sma2

# ticker = 'TSLA'
# start_date = '2022-05-20'
# end_date = '2025-05-20'
# max_profit, max_sma1, max_sma2 = optimal_sma(ticker, start_date, end_date)
# print(f"\n\nMax profit: ${max_profit}")
# print(f"SMA1: {max_sma1}")
# print(f"SMA2: {max_sma2}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********



Max profit: $490.72667694091797
SMA1: 9
SMA2: 15


In [ ]:
#Do above for a list of stock ticker
tickers = ['LLY', 'MSFT', 'SPY', 'CMG', 'MOD', 'TSLA']
start_date = '2022-05-20'
end_date = '2025-05-20'
optimal_smas_list = []
for ticker in tickers:
  max_profit, max_sma1, max_sma2 = optimal_sma(ticker, start_date, end_date)
  optimal_smas_list.append({'Stock': ticker, 'Est Profit': max_profit, 'opt_SMA1': max_sma1, 'opt_SMA2': max_sma2})

optimal_smas = pd.DataFrame(optimal_smas_list)
optimal_smas


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********